# ANOVA  - Lab

## Introduction

In this lab, you'll get some brief practice generating an ANOVA table (AOV) and interpreting its output. You'll also perform some investigations to compare the method to the t-tests you previously employed to conduct hypothesis testing.

## Objectives

In this lab you will: 

- Use ANOVA for testing multiple pairwise comparisons 
- Interpret results of an ANOVA and compare them to a t-test

## Load the data

Start by loading in the data stored in the file `'ToothGrowth.csv'`: 

In [41]:
# Your code here
import pandas as pd
import statsmodels.api as sm
from statsmodels.formula.api import ols
import scipy.stats as stats
import numpy as np

df = pd.read_csv('ToothGrowth.csv')
df.head()

,len,supp,dose
0,4.2,VC,0.5
1,11.5,VC,0.5
2,7.3,VC,0.5
3,5.8,VC,0.5
4,6.4,VC,0.5


## Generate the ANOVA table

Now generate an ANOVA table in order to analyze the influence of the medication and dosage:  

In [18]:
# Your code here
formula = 'len ~ C(supp) + dose'
lm = ols(formula, df).fit()
table = sm.stats.anova_lm(lm, typ=2)
print(table)

               sum_sq    df           F        PR(>F)
C(supp)    205.350000   1.0   11.446768  1.300662e-03
dose      2224.304298   1.0  123.988774  6.313519e-16
Residual  1022.555036  57.0         NaN           NaN


## Interpret the output

Make a brief comment regarding the statistics and the effect of supplement and dosage on tooth length: 

In [52]:
table['PR(>F)']['C(supp)']

0.0013006624739974717

In [25]:
# Your comment here
# Both Supplement & Dosage appear to have some influence on Tooth Length, with Dosage looking more significant

## Compare to t-tests

Now that you've had a chance to generate an ANOVA table, its interesting to compare the results to those from the t-tests you were working with earlier. With that, start by breaking the data into two samples: those given the OJ supplement, and those given the VC supplement. Afterward, you'll conduct a t-test to compare the tooth length of these two different samples: 

In [38]:
# Your code here
oj = df[df.supp == 'OJ'].len
vc = df[df.supp == 'VC'].len

Now run a t-test between these two groups and print the associated two-sided p-value: 

In [44]:
# Calculate the 2-sided p-value for a t-test comparing the two supplement groups
stats.ttest_ind(oj, vc, equal_var=False)

Ttest_indResult(statistic=1.91526826869527, pvalue=0.06063450788093387)

In [45]:
# t-critical
stats.t.ppf(1-0.05/2, 29)

2.045229642132703

In [46]:
# Comments
# (t-statistic < t-critical) AND (p-value > 0.05) --> fail to reject the Null Hypothesis

## A 2-Category ANOVA F-test is equivalent to a 2-tailed t-test!

Now, recalculate an ANOVA F-test with only the supplement variable. An ANOVA F-test between two categories is the same as performing a 2-tailed t-test! So, the p-value in the table should be identical to your calculation above.

> Note: there may be a small fractional difference (>0.001) between the two values due to a rounding error between implementations. 

In [53]:
# Your code here; conduct an ANOVA F-test of the oj and vc supplement groups.
# Compare the p-value to that of the t-test above. 
# They should match (there may be a tiny fractional difference due to rounding errors in varying implementations)
formula = 'len ~ C(supp)'
lm = ols(formula, df).fit()
table = sm.stats.anova_lm(lm, typ=2)
print(table)

               sum_sq    df         F    PR(>F)
C(supp)    205.350000   1.0  3.668253  0.060393
Residual  3246.859333  58.0       NaN       NaN


## Run multiple t-tests

While the 2-category ANOVA test is identical to a 2-tailed t-test, performing multiple t-tests leads to the multiple comparisons problem. To investigate this, look at the various sample groups you could create from the 2 features: 

In [73]:
lst = []
for group in df.groupby(['supp', 'dose'])['len']:
    group_name = group[0]
    data = group[1]
    lst.append(group_name)
    print(group_name)

('OJ', 0.5)
('OJ', 1.0)
('OJ', 2.0)
('VC', 0.5)
('VC', 1.0)
('VC', 2.0)


While bad practice, examine the effects of calculating multiple t-tests with the various combinations of these. To do this, generate all combinations of the above groups. For each pairwise combination, calculate the p-value of a 2-sided t-test. Print the group combinations and their associated p-value for the two-sided t-test.

In [96]:
# Your code here; reuse your t-test code above to calculate the p-value for a 2-sided t-test
# for all combinations of the supplement-dose groups listed above. 
# (Since there isn't a control group, compare each group to every other group.)
import itertools
combos = list(set(itertools.combinations(lst, 2)))

for c in combos:
    g1 = df[(df.supp == c[0][0]) & (df.dose == c[0][1])].len
    g2 = df[(df.supp == c[1][0]) & (df.dose == c[1][1])].len
    
    eql_var = True if np.isclose(np.var(g1), np.var(g2)) else False
    if len(g1) == len(g2):
        print(f'{c[0]} VS. {c[1]}: {stats.ttest_ind(g1, g2, equal_var=eql_var).pvalue}')

('OJ', 0.5) VS. ('VC', 0.5): 0.006358606764096813
('OJ', 0.5) VS. ('VC', 2.0): 7.196253524006043e-06
('VC', 0.5) VS. ('VC', 1.0): 6.811017702865016e-07
('OJ', 0.5) VS. ('OJ', 1.0): 8.784919055161479e-05
('OJ', 1.0) VS. ('VC', 0.5): 3.6552067303259103e-08
('OJ', 1.0) VS. ('VC', 2.0): 0.09652612338267014
('OJ', 2.0) VS. ('VC', 1.0): 2.3610742020468435e-07
('VC', 1.0) VS. ('VC', 2.0): 9.155603056638692e-05
('OJ', 0.5) VS. ('VC', 1.0): 0.04601033257637553
('OJ', 1.0) VS. ('VC', 1.0): 0.001038375872299884
('OJ', 0.5) VS. ('OJ', 2.0): 1.3237838776972294e-06
('OJ', 2.0) VS. ('VC', 0.5): 1.3621396478988818e-11
('OJ', 2.0) VS. ('VC', 2.0): 0.9638515887233756
('VC', 0.5) VS. ('VC', 2.0): 4.6815774144921145e-08
('OJ', 1.0) VS. ('OJ', 2.0): 0.039195142046244004


## Summary

In this lesson, you implemented the ANOVA technique to generalize testing methods to multiple groups and factors.